## Introduction

**                              Identify the number of channels open at each time point**

Many diseases, including cancer, are believed to have a contributing factor in common. Ion channels are pore-forming proteins present in animals and plants. They encode learning and memory, help fight infections, enable pain signals, and stimulate muscle contraction. If scientists could better study ion channels, which may be possible with the aid of machine learning, it could have a far-reaching impact.

When ion channels open, they pass electric currents. Existing methods of detecting these state changes are slow and laborious. Humans must supervise the analysis, which imparts considerable bias, in addition to being tedious. These difficulties limit the volume of ion channel current analysis that can be used in research. Scientists hope that technology could enable rapid automatic detection of ion channel current events in raw data.

The University of Liverpool’s Institute of Ageing and Chronic Disease is working to advance ion channel research. Their team of scientists have asked for your help. In this competition, you’ll use ion channel data to better model automatic identification methods. If successful, you’ll be able to detect individual ion channel events in noisy raw signals. The data is simulated and injected with real world noise to emulate what scientists observe in laboratory experiments.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import gc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Any results you write to the current directory are saved as output.
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, MinMaxScaler

from tqdm import tqdm

import lightgbm as lgb


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.utils import class_weight

from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import roc_curve, auc, accuracy_score, cohen_kappa_score
from sklearn.metrics import mean_squared_error, f1_score, confusion_matrix


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras


from tensorflow.keras.models import Sequential, Model

#from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import LSTM, Bidirectional, add, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Conv2DTranspose, AveragePooling1D, UpSampling1D
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation, TimeDistributed
from tensorflow.keras.layers import Multiply, Add, Concatenate, Flatten, Average, Lambda

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, Callback, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.constraints import unit_norm, max_norm
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.utils import np_utils

from tensorflow.keras import backend as K

In [ ]:
## utils

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024 ** 2 # just added 
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024 ** 2
    percent = 100 * (start_mem - end_mem) / start_mem
    print('Mem. usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(start_mem, end_mem, percent))
    return df

def get_stats(df):
    stats = pd.DataFrame(index=df.columns, columns=['na_count', 'n_unique', 'type', 'memory_usage'])
    for col in df.columns:
        stats.loc[col] = [df[col].isna().sum(), df[col].nunique(dropna=False), df[col].dtypes, df[col].memory_usage(deep=True, index=False) / 1024**2]
    stats.loc['Overall'] = [stats['na_count'].sum(), stats['n_unique'].sum(), None, df.memory_usage(deep=True).sum() / 1024**2]
    return stats

def print_header():
    print('col         conversion        dtype    na    uniq  size')
    print()
    
def print_values(name, conversion, col):
    template = '{:10}  {:16}  {:>7}  {:2}  {:6}  {:1.2f}MB'
    print(template.format(name, conversion, str(col.dtypes), col.isna().sum(), col.nunique(dropna=False), col.memory_usage(deep=True, index=False) / 1024 ** 2))

In [ ]:
import tensorflow as tf
import keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [ ]:
def display_set(df, column, n_sample, figsize ):
    f, ax1 = plt.subplots(nrows = 1, ncols = 1, figsize = figsize )
    sns.lineplot(x= df.index[::n_sample], y = df[column][::n_sample], ax=ax1)


In [ ]:
# Showing Confusion Matrix
# Thanks to https://www.kaggle.com/marcovasquez/basic-nlp-with-tensorflow-and-wordcloud
def plot_cm(y_true, y_pred, title):
    figsize=(14,14)
    y_pred = y_pred.astype(int)
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    plt.title(title)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)

## Load train and test datasets

**IMPORTANT: While the time series appears continuous, the data is from discrete batches of 50 seconds long 10 kHz samples (500,000 rows per batch).
In other words, the data from 0.0001 - 50.0000 is a different batch than 50.0001 - 100.0000, and thus discontinuous between 50.0000 and 50.0001.**

In [ ]:
PATH = '/kaggle/input/liverpool-ion-switching/'


def read_and_clean(file_name):
    return pd.read_csv(PATH + file_name)


train = read_and_clean('train.csv')
test = read_and_clean('test.csv')

train.head()

## Merge train and test and feature engineering

In [ ]:
## Most of the ideas have been taken from https://www.kaggle.com/gpreda/ion-switching-advanced-eda-and-prediction

In [ ]:
train['train'] = True
test['train'] = False
test['open_channels'] = np.nan

full = pd.concat([train, test], sort=False).reset_index(drop=True)
full['train'] = full['train'].astype('bool')
full = full.sort_values(by=['time']).reset_index(drop=True)

full.index = ((full.time * 10_000) - 1).values
full['batch'] = full.index // 50_000
full['batch_index'] = full.batch // 50_000
full['batch_index'] = full.index  - (full.batch * 50_000)
full['batch_slices'] = full['batch_index']  // 5_000
full['batch_slices2'] = full.apply(lambda r: '_'.join([str(r['batch']).zfill(3), str(r['batch_slices']).zfill(3)]), axis=1)

# 50_000 Batch Features
full['signal_batch_min'] = full.groupby('batch')['signal'].transform('min')
full['signal_batch_max'] = full.groupby('batch')['signal'].transform('max')
full['signal_batch_std'] = full.groupby('batch')['signal'].transform('std')
full['signal_batch_mean'] = full.groupby('batch')['signal'].transform('mean')
full['signal_batch_median'] = full.groupby('batch')['signal'].transform('median')
#full['signal_batch_skew'] = full.groupby('batch')['signal'].transform('skew')
#full['mean_abs_chg_batch'] = full.groupby(['batch'])['signal'].transform(lambda x: np.mean(np.abs(np.diff(x))))
#full['median_abs_chg_batch'] = full.groupby(['batch'])['signal'].transform(lambda x: np.median(np.abs(np.diff(x))))
#full['abs_max_batch'] = full.groupby(['batch'])['signal'].transform(lambda x: np.max(np.abs(x)))
#full['abs_min_batch'] = full.groupby(['batch'])['signal'].transform(lambda x: np.min(np.abs(x)))
#full['abs_mean_batch'] = full.groupby(['batch'])['signal'].transform(lambda x: np.mean(np.abs(x)))
#full['abs_median_batch'] = full.groupby(['batch'])['signal'].transform(lambda x: np.median(np.abs(x)))
#full['moving_average_batch_1000_mean'] = full.groupby(['batch'])['signal'].rolling(window=1000).mean().mean(skipna=True)

#full['range_batch'] = full['signal_batch_max'] - full['signal_batch_min']
#full['maxtomin_batch'] = full['signal_batch_max'] / full['signal_batch_min']
#full['abs_avg_batch'] = (full['abs_min_batch'] + full['abs_max_batch']) / 2

#add shifts
full['signal_shift+1'] = full.groupby(['batch']).shift(1)['signal']
full['signal_shift-1'] = full.groupby(['batch']).shift(-1)['signal']
#full['signal_shift+2'] = full.groupby(['batch']).shift(2)['signal']
#full['signal_shift-2'] = full.groupby(['batch']).shift(-2)['signal']

#full['signal_shift+1_5k'] = full.groupby(['batch_slices2']).shift(1)['signal']
#full['signal_shift-1_5k'] = full.groupby(['batch_slices2']).shift(-1)['signal']
#full['signal_shift+2_5k'] = full.groupby(['batch_slices2']).shift(2)['signal']
#full['signal_shift-2_5k'] = full.groupby(['batch_slices2']).shift(-2)['signal']

#full['abs_max_signal_shift+1_5k'] = full['signal_shift+1_5k'].transform(lambda x: np.max(np.abs(x)))
#full['abs_max_signal_shift-1_5k'] = full['signal_shift-1_5k'].transform(lambda x: np.max(np.abs(x)))

In [ ]:
window_sizes = [10, 50, 500, 1000, 5000]
for window in window_sizes:
    full["rolling_mean_" + str(window)] = full['signal'].rolling(window=window).mean()
    full["rolling_std_" + str(window)] = full['signal'].rolling(window=window).std()
    full["rolling_var_" + str(window)] = full['signal'].rolling(window=window).var()
    full["rolling_min_" + str(window)] = full['signal'].rolling(window=window).min()
    full["rolling_max_" + str(window)] = full['signal'].rolling(window=window).max()
    
    full["rolling_min_max_ratio_" + str(window)] = full["rolling_min_" + str(window)] / full["rolling_max_" + str(window)]
    full["rolling_min_max_diff_" + str(window)] = full["rolling_max_" + str(window)] - full["rolling_min_" + str(window)]
    
    a = (full['signal'] - full['rolling_min_' + str(window)]) / (full['rolling_max_' + str(window)] - full['rolling_min_' + str(window)])
    full["norm_" + str(window)] = a * (np.floor(full['rolling_max_' + str(window)]) - np.ceil(full['rolling_min_' + str(window)]))
    
full = full.replace([np.inf, -np.inf], np.nan)    
full.fillna(0, inplace=True)

In [ ]:
FEATURES = [f for f in full.columns if f not in ['open_channels','index','time','train', 'batch_slices', 'batch_slices2']]
print(f"Features: {len(FEATURES)}")
print([f for f in FEATURES])

In [ ]:
get_stats(full)

## Display train and test signals

In [ ]:
DATA_BATCH_SIZE = 500000

TRAIN_SAMPLE_RATE = 100
TRAIN_BATCH_SIZE = int(len(train)/TRAIN_SAMPLE_RATE)

f, ax1 = plt.subplots(nrows = 1, ncols = 1, figsize = (20,4))
sns.lineplot(data=train.signal[::TRAIN_SAMPLE_RATE], ax=ax1, hue="size", size="size")
sns.lineplot(data=train.open_channels[::TRAIN_SAMPLE_RATE], ax=ax1, hue="size", size="size")
ax1.set_title(f'Full train signal')

f, ax1 = plt.subplots(nrows = 1, ncols = 1, figsize = (10,4))
sns.lineplot(data=test.signal[::TRAIN_SAMPLE_RATE], ax=ax1, hue="size", size="size")
ax1.set_title(f'Full test signal')


## By batch

In [ ]:
f, axes = plt.subplots(nrows = 2, ncols = 5, figsize = (26,12))
for i in range(10):
    XX = train.signal[i*DATA_BATCH_SIZE:(i+1)*DATA_BATCH_SIZE + 1]
    yy = train.open_channels[i*DATA_BATCH_SIZE:(i+1)*DATA_BATCH_SIZE + 1]
    sns.scatterplot(data=XX[::TRAIN_SAMPLE_RATE], ax=axes[int(i/5), i%5], hue="size", size="size")
    sns.scatterplot(data=yy[::TRAIN_SAMPLE_RATE], ax=axes[int(i/5), i%5], hue="size", size="size")
    axes[int(i/5), i%5].set_title(f'Train Batch# {i+1}')
    
f, axes = plt.subplots(nrows = 1, ncols = 5, figsize = (26,6))
for i in range(4):
    XX = test.signal[i*DATA_BATCH_SIZE:(i+1)*DATA_BATCH_SIZE + 1]
    sns.scatterplot(data=XX[::TRAIN_SAMPLE_RATE], ax=axes[i], hue="size", size="size")
    axes[i].set_title(f'Test Batch# {i+1}')


## Open channels value count

In [ ]:
f, ax = plt.subplots(figsize=(15, 6))
sns.countplot(x="open_channels", data=train, ax=ax)

## Distributions for open channels for each batch

In [ ]:
sns.set(style="whitegrid")

f, axes = plt.subplots(nrows = 2, ncols = 5, figsize = (26,12))
for i in range(10):
    y = pd.DataFrame()
    sns.countplot( x = train.open_channels[i*DATA_BATCH_SIZE:(i+1)*DATA_BATCH_SIZE + 1], ax=axes[int(i/5), i%5])
    axes[int(i/5), i%5].set_title(f'Train Batch# {i+1}')

## Compute correlation matrix

In [ ]:
corr = full[::1000].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
train_df = full.query('train').copy()
test_df = full.query('not train').copy()
train_df['open_channels'] = train_df['open_channels'].astype(int)

X_all_train = train_df[FEATURES]
X_all_test = test_df[FEATURES]
y_all_train = train_df['open_channels'].values

## reduce amount of data to speed things up
X_train = X_all_train[::10]
y_train = y_all_train[::10]
X_test = X_all_test[::10] ## this is just for display purposes


del full
del train_df
del test_df
gc.collect()

print(f'Original sizes: X_all_train: {X_all_train.shape}, y_all_train: {y_all_train.shape}, X_all_test: {X_all_test.shape}' )
print(f'Reduced train sizes: X_train: {X_train.shape}, y_train: {y_all_train.shape}, X_test: {X_test.shape}' )

## LSTM model

In [ ]:
def baseline_model(input_shape, units = 32, max_channels = 11, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, input_shape=(input_shape[1], input_shape[2]), return_sequences=True))
    model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units))
    model.add(Dense(units))
    model.add(Dense(max_channels, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc', f1])
    return model

In [ ]:
## it seems LSTM model will not converge with all generated features (at least for me :) 
## so, select some that work

RANDOM_SEED = 42

LSTM_FEATURES = ['signal', 'rolling_min_max_diff_5000', 'signal_batch_mean', 'norm_5000', 'batch_index', 
                 'rolling_max_5000', 'norm_1000', 'signal_shift-1', 'signal_shift+1']

X = X_train[LSTM_FEATURES]
y = y_train

X = X[::10]
y = y[::10]


## 
#X.signal = (X.signal-X.signal.mean())/X.signal.std()

## reshape for LSTM
X = X.values.reshape(-1,len(LSTM_FEATURES),1)
## using categorical_crossentropy
yy = to_categorical(y, num_classes=11)

train_idx, val_idx = train_test_split(np.arange(X.shape[0]), random_state = RANDOM_SEED, test_size = 0.2)

X_t = X[train_idx] 
y_t = yy[train_idx] 
X_v = X[val_idx]
y_v = yy[val_idx]

In [ ]:
BATCH_SIZE = 64
EPOCHS = 20

es = EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose=1)
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.000001, verbose=1)

adam = Adam(0.01)

model = baseline_model(X_t.shape, optimizer=adam)
history = model.fit( X_t, y_t, validation_data=(X_v, y_v), callbacks=[es,lr],
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, 
                    shuffle=False, workers=8, use_multiprocessing=True )

In [ ]:
_, ax1 = plt.subplots(nrows = 1, ncols = 1, figsize = (16,8))
sns.lineplot(data=np.asarray(history.history['loss']), label='loss', ax=ax1)
sns.lineplot(data=np.asarray(history.history['val_loss']), label='val_loss', ax=ax1)
#sns.lineplot(data=np.asarray(history.history['acc']), label='acc', ax=ax1)
#sns.lineplot(data=np.asarray(history.history['val_acc']), label='val_acc', ax=ax1)
sns.lineplot(data=np.asarray(history.history['f1']), label='f1', ax=ax1)
sns.lineplot(data=np.asarray(history.history['val_f1']), label='val_f1', ax=ax1)

## Evaluate model

In [ ]:
y_pred = np.argmax(model.predict(X_v), axis=1).reshape(-1)
yhat = y[val_idx]

print("SCORE_oldmetric: ", cohen_kappa_score(yhat, y_pred, weights="quadratic"))
print("SCORE_newmetric: ", f1_score(yhat, y_pred, average="macro"))

## Making predictions

In [ ]:
y_pred = np.argmax(model.predict(X_all_test[LSTM_FEATURES].values.reshape(-1,len(LSTM_FEATURES), 1)), axis=1).reshape(-1)

In [ ]:
sub = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv", dtype={'time':str})
sub.open_channels = np.array(np.round(y_pred,0), np.int)
sub.to_csv("submission.csv",index=False)

In [ ]:
sub.head(25)